In [ ]:
!pip install einops
!pip install vit_pytorch

In [ ]:
from rl_env import *
from dgn_agent import *
from train_test_agent import *
from vit import *

import matplotlib as plt
import os
import pandas as pd

import torchvision

In [ ]:
models_save_path        = "./model"
results_save_path_agent = "./result"
dataset_path            = "./dataset"

# Seed and Device Setting

In [ ]:
def set_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

set_seed(42)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

# Dataset

## Dataset Parameters

In [ ]:
batch_size = 128
img_size = 32
dataset_name = "CIFAR10_BIG_"

## Train, validation and test

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.Resize(img_size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_validation = transforms.Compose([
    transforms.Resize(img_size),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


# Prepare/download dataset
trainset = torchvision.datasets.CIFAR10(root=dataset_path, train=True, download=True, transform=transform_train)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)

validationset = torchvision.datasets.CIFAR10(root=dataset_path, train=False, download=True, transform=transform_validation)

dataset_size = len(validationset)
validation_size = int(0.95 * dataset_size)
test_size = dataset_size - validation_size

validationset, testset = data.random_split(validationset, [validation_size, test_size])

validation_loader = torch.utils.data.DataLoader(validationset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

classes = trainset.classes
print(classes)

# Training 

## Parameter Initialization

In [ ]:
buffer_batch_size = 8
buffer_size = 64

gamma = 0.95

eps_start = 1
eps_end = 0.01
eps = eps_start
eps_decay = 20000

lr = 0.01

tau = 0.1
update_every = 2

get_reward_every = 10

pretrained = True

max_reward = 10
alpha = 0.2 # Where alpha defines weight of loss reward, and 1-alpha defines weight of patch (time) reward
# loss_weight = max_reward*(alpha)  # TODO THIS ONE TOO
# time_weight = max_reward*(1-alpha) # TODO PLAY WITH THESE VALUES A LOT
loss_weight = alpha
time_weight = 1-alpha
    
# Input image to DQN agent's  Q-network
dqn_img_w = 32 # for CIFAR, lowres=16, fullres=32
dqn_input_channels = 3
dqn_input_size = dqn_img_w*dqn_img_w*dqn_input_channels # input to DQN (mlp type)

In [ ]:
# This is the number of patches along the width/height of the square image.
patch_width = 4
# patch_size = int(img_size/patch_width)
patch_size = patch_width # This is what you get when you work with someone else's code and don't want to rewrite everything.


total_patches = int((img_size/patch_width)**2)
print(total_patches)
n_patch_selected = int(total_patches*(40/64)) # From the paper


att_dim = 128

epochs = 100
learning_rate = 0.001

ViTnet = SimpleAgentViT(
    image_size = img_size,
    patch_size = patch_size,
    num_classes = len(classes),
    dim = att_dim,
    depth = 6,
    heads = 16,
    mlp_dim = 512
)

# Sposta il modello sulla GPU (se disponibile)
ViTnet.to(device)

# definiamo l'ottimizzatore
optimizer = optim.Adam(ViTnet.parameters(), lr=learning_rate)

## Create Environment and Training Agent

In [ ]:
#env = ViTEnv(ViTnet, total_patches, optimizer, loss_weight, time_weight, device, n_patch_selected)
env = ViTEnv(ViTnet, total_patches, optimizer, loss_weight, time_weight, device, n_patch_selected, total_epochs=epochs)

In [ ]:
model = TrainingTestingAgent(epochs = epochs,
                             model = ViTnet,
                             get_reward_every = get_reward_every,
                             buffer_batch_size = buffer_batch_size,
                             batch_size = batch_size,
                             env = env,
                             att_dim = att_dim,
                             n_patches = total_patches,
                             buffer_size = buffer_size,
                             gamma = gamma,
                             tau = tau,
                             update_every = update_every,
                             lr = lr,
                             eps_end = eps_end,
                             eps_start = eps_start,
                             eps_decay = eps_decay,
                             train_loader = train_loader,
                             validation_loader = validation_loader,
                             device = device,
                             dqn_input_size=dqn_input_size,
                             pretrained=pretrained)

## Main Training Loop

In [ ]:
initial = time.time()
step_reward, selected_patch = model.train_test()
print(f'Total Time: {time.time()-initial}')

### Training and Validation Metrics

In [ ]:
results_train = model.train_info()
train_loss = results_train['train_loss']
train_time = results_train['train_time']

In [ ]:
results_validation = model.validation_info()
validation_loss = results_validation['validation_loss']
validation_acc = results_validation['validation_acc']

In [ ]:
save_name="highres_pretrained-frozen-cnn_default-configs_4patch_rebalanceR"

df = pd.DataFrame(results_train)

df.to_csv(f'{results_save_path_agent}{dataset_name}_train_lowres_dr_e100_{save_name}.csv', index=False)

In [ ]:
df = pd.DataFrame(results_validation)

df.to_csv(f'{results_save_path_agent}{dataset_name}_validation_lowres_dr_e100_{save_name}.csv', index=False)

In [ ]:
model_name = "high-Res Pretrained frozen CNN (Default Configs), 4x4 patches, ReBalance Reward"

plt.plot(range(len(train_loss)), train_loss, label='Train Loss')

plt.legend()
plt.title('Loss over time: {}'.format(model_name))
plt.xlabel('Iteration')
plt.ylabel('Loss')

plt.show()

In [ ]:
plt.plot(range(len(validation_loss)), validation_loss, label='Validation Loss')

plt.legend()
plt.title('Loss over time: {}'.format(model_name))
plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.show()

In [ ]:
x = range(len(validation_acc))
y = validation_acc

plt.plot(x, y, label='Validation Accuracy')

plt.legend()
plt.title('Accuracy over time: {}'.format(model_name))
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

plt.show()

## Evaluation for different Heuristics

### Training Set: All Patches

In [ ]:
evaluate_agent(ViTnet, train_loader, device, ViTnet, model)

### Training Set: Agent-Selected Patches

In [ ]:
# time-dependent reward component
evaluate_agent(ViTnet, train_loader, device, ViTnet, model, mode = "agent")

### Training Set: Random Patches

In [1]:
# time-dependent reward component
evaluate_agent(ViTnet, train_loader, device, ViTnet, model, mode = "random")

NameError: name 'evaluate_agent' is not defined

### Validation Set: All Patches

In [ ]:
evaluate_agent(ViTnet, test_loader, device, ViTnet, model)

### Validation Set: Agent-Selected Patches

In [ ]:
evaluate_agent(ViTnet, test_loader, device, mode = "agent")

### Validation Set: Random Patches

In [ ]:
evaluate_agent(ViTnet, test_loader, device, mode = "random")